In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from matplotlib.colors import ListedColormap
X = X[['MA20', 'RSI']]  # Change these to any two features you want to visualize

# Step 2: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Step 3: Initialize and train the KNN classifier
model = KNeighborsClassifier(n_neighbors=1, weights='uniform', algorithm='auto', leaf_size=30, p=1, metric='minkowski')
model.fit(X_train, y_train)

# Step 4: Create a grid to plot decision boundaries
x_min, x_max = X['MA20'].min() - 1, X['MA20'].max() + 1
y_min, y_max = X['RSI'].min() - 1, X['RSI'].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))

# Step 5: Predict the class for each point in the mesh
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Step 6: Plot decision boundaries
plt.figure(figsize=(12, 6))
plt.contourf(xx, yy, Z, alpha=0.3, cmap=ListedColormap(('red', 'blue')))

# Step 7: Plot the original data points
scatter = plt.scatter(X['MA20'], X['RSI'], c=y, edgecolor='k', s=50, cmap=ListedColormap(('red', 'blue')))

# Step 8: Add labels and legend
plt.title('KNN Classification Plot with Selected Features (MA20 and RSI)')
plt.xlabel('MA20')
plt.ylabel('RSI')
plt.colorbar(scatter, label='Classes')

In [ ]:
# X, y = make_moons(noise=0.3, random_state=0)
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y.astype(str), test_size=0.25, random_state=0)
# X, Y = make_classification(n_samples=2000, n_features=2, n_informative=2, n_redundant=0, random_state=42)
trace_specs = [
    [X_train, y_train, '0', 'Train', 'square'],
    [X_train, y_train, '1', 'Train', 'circle'],
    [X_test, y_test, '0', 'Test', 'square-dot'],
    [X_test, y_test, '1', 'Test', 'circle-dot']
]

fig = go.Figure(data=[
    go.Scatter(
        x=X[y==label, 2], y=X[y==label, 3],
        name=f'{split} Split, Label {label}',
        mode='markers', marker_symbol=marker
    )
    for x, y, label, split, marker in trace_specs
])
fig.update_traces(
    marker_size=12, marker_line_width=1.5,
    marker_color="lightyellow"
)
fig.show()

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from scipy.signal import argrelextrema

# Assume 'data' is your DataFrame with OHLC data and a DateTime index
# Example structure: data['Open'], data['High'], data['Low'], data['Close']

# Parameters for finding local extrema
order = 10

# Find the indices of local maxima and minima
max_idx = argrelextrema(data['Close'].values, np.greater, order=order)[0]
min_idx = argrelextrema(data['Close'].values, np.less, order=order)[0]

# Create the candlestick chart
fig = go.Figure(data=[go.Candlestick(x=data.index,
                                     open=data['Open'],
                                     high=data['High'],
                                     low=data['Low'],
                                     close=data['Close'])])

# Add scatter plot for maxima (v-shaped)
fig.add_trace(go.Scatter(x=data.iloc[max_idx].index, 
                         y=data.iloc[max_idx]['Close'], 
                         mode='markers', 
                         marker=dict(color='red', symbol='triangle-down', size=10),
                         name='Maxima'))

# Add scatter plot for minima (^ shaped)
fig.add_trace(go.Scatter(x=data.iloc[min_idx].index, 
                         y=data.iloc[min_idx]['Close'], 
                         mode='markers', 
                         marker=dict(color='green', symbol='triangle-up', size=10),
                         name='Minima'))

# Update layout for the plot
fig.update_layout(
    title='Candlestick Chart with Local Maxima and Minima',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False  # Optional: Removes the date range slider below the chart
)

# Show the figure
fig.show()


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

# Step 1: Load EUR/USD historical data
data = pd.read_csv('15M_EURUSD.csv')  # Replace with your own file

# Step 2: Create simple features
data['Price_diff'] = data['Close'].diff()  # Price difference (day-to-day)
data['MA_5'] = data['Close'].rolling(window=5).mean()  # 5-day moving average

# Step 3: Define target (1 = price up, -1 = price down)
data['Target'] = np.where(data['Close'].shift(-1) > data['Close'], 1, -1)

# Drop NaN values
data = data.dropna()

# Step 4: Define features and target
X = data[['Price_diff', 'MA_5']]
y = data['Target']

# Step 5: Train KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)  # k = 3 for simplicity
knn.fit(X, y)

# Step 6: Predict the direction (buy/sell signals)
data['Signal'] = np.where(knn.predict(X) == 1, 'Buy', 'Sell')

# Step 7: Plot the signals
plt.figure(figsize=(12, 6))
plt.plot(data['Close'], label='EUR/USD Close Price', color='blue')

# Plot Buy signals
plt.plot(data[data['Signal'] == 'Buy'].index, 
         data[data['Signal'] == 'Buy']['Close'], 
         '^', markersize=10, color='green', label='Buy Signal')

# Plot Sell signals
plt.plot(data[data['Signal'] == 'Sell'].index, 
         data[data['Signal'] == 'Sell']['Close'], 
         'v', markersize=10, color='red', label='Sell Signal')

# Add labels and legend
plt.title('KNN Buy/Sell Signals on EUR/USD')
plt.xlabel('Date')
plt.ylabel('EUR/USD Price')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from matplotlib.colors import ListedColormap

# Step 1: Generate synthetic EUR/USD-like data (2D for visualization)
# This is just for the sake of visualization; in practice, you'd load your actual EUR/USD data
X, y = make_classification(n_samples=100, n_features=2, n_classes=2, n_clusters_per_class=1, n_redundant=0, random_state=42)

# Step 2: Train KNN classifier
k = 3  # Number of neighbors
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X, y)

# Step 3: Create a grid to plot decision boundaries
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1), np.arange(y_min, y_max, 0.1))

# Step 4: Predict the class for each point in the mesh
Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Step 5: Plot decision boundaries
plt.figure(figsize=(12, 6))
plt.contourf(xx, yy, Z, alpha=0.3, cmap=ListedColormap(('red', 'blue')))

# Step 6: Plot the original data points
scatter = plt.scatter(X[:, 0], X[:, 1], c=y, edgecolor='k', s=50, cmap=ListedColormap(('red', 'blue')))

# Step 7: Add labels and legend
plt.title(f'KNN Classification Plot with k={k}')
plt.xlabel('Feature 1 (e.g., Price Diff)')
plt.ylabel('Feature 2 (e.g., MA)')
plt.colorbar(scatter, label='Classes')
plt.grid(True)
plt.show()

